In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments
import torch
import os
import pandas as pd
from tqdm import tqdm
from datasets import Dataset
from fuzzywuzzy import fuzz
import evaluate
import csv
import numpy as np
import torch.nn.functional as F
from sklearn.metrics import top_k_accuracy_score, ndcg_score
import random

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/tripadvisor/splits/train.jsonl"
df_recommender_train = pd.read_json(input_file, lines=True)
for _, row in df_recommender_train.iterrows():
    row["recommended_place"]["place_name"] = row["recommended_place"]["place_name"].lower()

In [5]:
df_recommender_train

,user_id,user_previous_interactions,recommended_place,negative_recommended_place,turns
0,,[],"{'place_name': 'fairmont copley plaza, boston'...","[{'place_name': 'The Eliot Hotel', 'business_i...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1,,[],{'place_name': 'hawthorn suites - austin south...,[{'place_name': 'Hyatt Place Austin/Arboretum'...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
2,,[],"{'place_name': 'warwick denver', 'business_id'...",[{'place_name': 'Hilton Garden Inn Denver Tech...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
3,,[],{'place_name': 'hilton phoenix tapatio cliffs ...,"[{'place_name': 'Hyatt Regency Phoenix', 'busi...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
4,,[{'place_name': 'Sonesta Select Dallas Central...,"{'place_name': 'kimpton banneker hotel', 'busi...","[{'place_name': 'The Westin Georgetown, Washin...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
...,...,...,...,...,...
9240,D99CCAF4543D579BF12D8138C1B977EE,"[{'place_name': 'The Boxer Boston', 'business_...","{'place_name': 'omni shoreham hotel', 'busines...","[{'place_name': 'Hamilton Hotel', 'business_id...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
9241,B1C728BCAC3EDC44731838495755AB84,[{'place_name': 'JW Marriott San Francisco Uni...,{'place_name': 'springhill suites austin north...,[{'place_name': 'La Quinta Inn by Wyndham Aust...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
9242,6562BBD4EA770FE84E579622F68FA181,[{'place_name': 'The Westin Detroit Metropolit...,"{'place_name': 'stanford court san francisco',...",[{'place_name': 'Holiday Inn Express & Suites ...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
9243,FBE6337895DDB52EB0F7D3EE4EF114A1,[{'place_name': 'SpringHill Suites Seattle Dow...,{'place_name': 'club quarters hotel in washing...,[{'place_name': 'Residence Inn Washington Nati...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."


In [6]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/tripadvisor/splits/val.jsonl"
df_recommender_validation = pd.read_json(input_file, lines=True)
for _, row in df_recommender_validation.iterrows():
    row["recommended_place"]["place_name"] = row["recommended_place"]["place_name"].lower()

In [7]:
df_recommender_validation

,user_id,user_previous_interactions,recommended_place,negative_recommended_place,turns
0,449537356F925E964EAB75E5E248E3C9,"[{'place_name': 'Swissotel Chicago', 'business...","{'place_name': 'le meridien philadelphia', 'bu...",[{'place_name': 'Hilton Philadelphia at Penn's...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1,9670873F4A6B68975E8F42C780351663,"[{'place_name': 'Mediterranean Inn', 'business...",{'place_name': 'days inn by wyndham washington...,[{'place_name': 'Sofitel Washington DC Lafayet...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
2,1358E6157B951E9C29463FE7DB1D6FB9,[{'place_name': 'Hyatt Regency DFW Internation...,"{'place_name': 'the mosser', 'business_id': 81...","[{'place_name': 'San Francisco Proper', 'busin...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
3,99ABE6D55851EE4059907581FC98A69D,"[{'place_name': 'Hilton Boston Park Plaza', 'b...","{'place_name': 'xv beacon hotel', 'business_id...","[{'place_name': 'Hotel Indigo Boston Garden, a...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
4,931A3EAA5FB329245F5EB12D6E93BF1F,[{'place_name': 'Hyatt Regency San Francisco D...,"{'place_name': 'four seasons hotel seattle', '...","[{'place_name': 'Inn at the Market', 'business...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
...,...,...,...,...,...
1979,49D9199B68F455F5C40F8D9414515471,"[{'place_name': 'Sofitel New York', 'business_...","{'place_name': 'the iroquois new york', 'busin...","[{'place_name': 'Sofitel New York', 'business_...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1980,E4BD9A4CD7872825F3585ECFFF4074B7,"[{'place_name': 'Hyatt Regency Phoenix', 'busi...","{'place_name': 'la cantera resort & spa', 'bus...",[{'place_name': 'San Antonio Marriott Rivercen...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1981,26697D4B49A57BA92334EF8326DC6F76,"[{'place_name': 'The Bellevue Hotel', 'busines...",{'place_name': 'the royal sonesta washington d...,[{'place_name': 'InterContinental The Willard ...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1982,A64AF839382CBFD4F7653A7DD3228246,[{'place_name': 'Crowne Plaza Memphis Downtown...,"{'place_name': 'the westin phoenix downtown', ...","[{'place_name': 'The Clarendon Hotel and Spa',...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."


In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name_or_path = "google/flan-t5-base")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base", additional_special_tokens=["computer:", "human:"])
model.resize_token_embeddings(len(tokenizer))
IGNORE_INDEX = -100

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
prompt_train = []
recommend_train = []
not_founds = 0

for _, row in tqdm(df_recommender_train.iterrows(), total=len(df_recommender_train)):
    prompt = ""
    found = False
    recommended = row["recommended_place"]["place_name"]
    
    for index, turn in enumerate(row["turns"]):
        if "COMPUTER" in turn:
            computer = turn["COMPUTER"]
        
            if fuzz.partial_ratio(recommended, computer.lower()) >= 95:
                prompt += "computer: I would recommend the "
                prompt_train.append(prompt)
                recommend_train.append(recommended)
                found = True
                break
            else:
                prompt += "computer: "+ computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1

print(f"Could not find {not_founds}")
print(f"len(prompt_train): {len(prompt_train)}")
print(f"len(recommend_train): {len(recommend_train)}")

            
            
prompt_encodings = tokenizer(prompt_train, padding='max_length', max_length=1024, truncation=True, return_tensors='pt')
recommend_encodings = tokenizer(recommend_train, padding='max_length', max_length=32, truncation=True, return_tensors='pt')

labels = recommend_encodings['input_ids']
labels[labels == tokenizer.pad_token_id] = IGNORE_INDEX

dataset = {
    'input_ids': prompt_encodings['input_ids'],
    'attention_mask': prompt_encodings['attention_mask'],
    'labels': labels,
}
dataset_train = Dataset.from_dict(dataset)

  0%|          | 0/9245 [00:00<?, ?it/s]

100%|██████████| 9245/9245 [00:49<00:00, 187.18it/s]


Could not find 6
len(prompt_train): 9239
len(recommend_train): 9239


In [9]:
prompt_validation = []
recommend_validation = []
not_founds = 0

for _, row in tqdm(df_recommender_validation.iterrows(), total=len(df_recommender_validation)):
    prompt = ""
    found = False
    recommended = row["recommended_place"]["place_name"]
    
    for index, turn in enumerate(row["turns"]):
        if "COMPUTER" in turn:
            computer = turn["COMPUTER"]
            
            if fuzz.partial_ratio(recommended, computer.lower()) >= 95:
                prompt += "computer: I would recommend the "
                prompt_validation.append(prompt)
                recommend_validation.append(recommended)
                found = True
                break
            else:
                prompt += "computer: "+ computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1
        
print(f"Could not find {not_founds}")
print(f"len(prompt_validation): {len(prompt_validation)}")
print(f"len(recommend_validation): {len(recommend_validation)}")
            
            
prompt_encodings = tokenizer(prompt_validation, padding='max_length', max_length=1024, truncation=True, return_tensors='pt')
recommend_encodings = tokenizer(recommend_validation, padding='max_length', max_length=32, truncation=True, return_tensors='pt')

labels = recommend_encodings['input_ids']
labels[labels == tokenizer.pad_token_id] = IGNORE_INDEX

dataset = {
    'input_ids': prompt_encodings['input_ids'],
    'attention_mask': prompt_encodings['attention_mask'],
    'labels': labels,
}
dataset_validation = Dataset.from_dict(dataset)

100%|██████████| 1984/1984 [00:11<00:00, 168.59it/s]


Could not find 3
len(prompt_validation): 1981
len(recommend_validation): 1981


In [10]:
def data_collator(batch):
    input_ids, attention_mask, labels,  = [], [], []
    for sample in batch:
        input_ids.append(sample['input_ids'])
        attention_mask.append(sample['attention_mask'])
        labels.append(sample['labels'])
    max_encoder_len = max(sum(x) for x in attention_mask)
    max_decoder_len = max(sum([0 if item == IGNORE_INDEX else 1 for item in x]) for x in labels)
    return {
        'input_ids': torch.tensor(input_ids)[:, :max_encoder_len],
        'attention_mask': torch.tensor(attention_mask)[:, :max_encoder_len],
        'labels': torch.tensor(labels)[:, :max_decoder_len]
    }

In [11]:
training_args = TrainingArguments(
    output_dir="/u/spa-d4/grad/mfe261/Projects/MobileConvRec/models/new_models/tripadvisor/T5_recommender",
    num_train_epochs=5,
    # logging_steps=500,
    # logging_dir=self.cfg.logging_dir,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy="steps",
    evaluation_strategy="steps",
    save_steps=0.3,#self.cfg.save_steps,
    eval_steps=0.3, #self.cfg.eval_steps,
    save_total_limit=3,
    gradient_accumulation_steps=3, #gradient_accumulation_steps,
    per_device_train_batch_size=4, #train_batch_size,
    per_device_eval_batch_size=4, #self.cfg.eval_batch_size,
    warmup_steps=100,
    weight_decay=0.01,
    # dataloader_drop_last=True,
    disable_tqdm=False,
    push_to_hub=False
)

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset_train,
        eval_dataset=dataset_validation,
        data_collator=data_collator,
    )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
trainer.train() # resume_from_checkpoint=True
trainer.save_model()

Step,Training Loss,Validation Loss
1155,1.234100,1.140860
2310,0.812000,0.930314
3465,0.658600,0.877899


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


# Load the model and test it on the test dataset

In [3]:
torch.cuda.empty_cache()

In [3]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/tripadvisor/splits/test.jsonl"
df_recommender_test = pd.read_json(input_file, lines=True)
for _, row in df_recommender_test.iterrows():
    row["recommended_place"]["place_name"] = row["recommended_place"]["place_name"].lower()

In [4]:
apps_training_path = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/tripadvisor/trip_df.csv"

all_apps = []
with open(apps_training_path, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        all_apps.append(row["name"].lower())
        
all_apps = list(set(all_apps))

In [5]:
all_apps

['crowne plaza houston - brookhollow',
 'motel 6 phoenix - northern avenue',
 'hilton garden inn washington, dc downtown',
 'comfort suites - columbus / keim circle',
 'wingate by wyndham jacksonville / mayo clinic area',
 'courtyard by marriott phoenix west/avondale',
 'the stratford',
 'extended stay deluxe - fort worth - fossil creek',
 'holiday inn nyc - manhattan 6th avenue',
 'homewood suites fort worth west at cityview',
 'courtyard by marriott san antonio downtown/market square',
 'homewood suites phoenix north - happy valley',
 'holiday inn fort worth north-fossil creek',
 'baymont inn austin north / austin highland mall',
 'omni fort worth hotel',
 'fairfield inn philadelphia airport',
 'the oxford hotel',
 'hampton inn houston hobby airport',
 'the jefferson, washington dc',
 'intercontinental chicago',
 'la quinta inn dallas lbj/central',
 'la quinta inn & suites austin airport',
 'loews denver hotel',
 'fairfield inn & suites - phoenix',
 'hampton inn indianapolis - ne / c

In [15]:
def candidate_creator(row):
    np.random.seed(row.name)
    selected_values = np.random.choice(np.setdiff1d(all_apps, [row["recommended_place"]["place_name"]]), 24, replace=False)
    random_position = np.random.randint(0, len(selected_values) + 1)
    
    return np.insert(selected_values, random_position, row["recommended_place"]["place_name"]) 

df_recommender_test['candidate'] = df_recommender_test.apply(lambda row: candidate_creator(row), axis=1)

In [ ]:
prompt_test = []
recommend_test = []
candidate_books = []
true_candidate_indexes = []
not_founds = 0
for _, row in df_recommender_test.iterrows():
    candidates = []
    for index, candidate_book in enumerate(row["candidate"].tolist()):
        candidates.append(candidate_book)
        if candidate_book == row["recommended_place"]["place_name"]:
            true_candidate_index = index
    prompt = ""
    
    found = False
    recommended = row["recommended_place"]["place_name"] 
    
    for index, turn in enumerate(row["turns"]):
        computer = turn["COMPUTER"]
        
        if fuzz.partial_ratio(recommended, computer.lower()) >= 95:
            prompt += "computer: I would recommend the "
            prompt_test.append(prompt)
            recommend_test.append(recommended)
            candidate_books.append(candidates)
            true_candidate_indexes.append(true_candidate_index)
            found = True
            break
        else:
            prompt += "computer: "+ computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1

print(f"Could not find {not_founds}")
print(f"Number of prompt: {len(prompt_test)}")
print(f"Number of generations: {len(recommend_test)}")
print(f"Number of candidate apps: {len(candidate_books)}")
print(f"Number of true candidate indexes: {len(true_candidate_indexes)}")

In [8]:
prompt_test[10]

'computer: Hi! How can I help you today?\nhuman: I\'m searching for hotels with \'River View\' or \'Marina View\' in Boston, Massachusetts. Any ideas?\ncomputer: Can you tell me which features you’re eager to see?\nhuman: I\'m looking for places that prioritize stylish design.\ncomputer: Do you need a gym as part of the hotel amenities?\nhuman: Yes, I would like the fitness center to be near the hotel.\ncomputer: Are you looking for a beautiful location?\nhuman: I usually look for hotels with a great location rating.\ncomputer: Based on what you\'ve told me, I recommend giving "XV Beacon Hotel" at 15 Beacon Street, Boston, MA 02108-2902 a try.\nhuman: I noticed that the hotel you recommended has some negative feedback. Can you tell me more about those issues? Specifically, how dark is the decor, and is the staff really unfriendly?\ncomputer: Referring to a number of user reviews, it’s mentioned that the decor is quite dark, and the staff has been noted as unfriendly by several guests.\

In [9]:
recommend_test[10]

'battery wharf hotel, boston waterfront'

In [10]:
candidate_books[10]

['w seattle',
 'the westin st. francis',
 'hampton inn & suites phoenix north / happy valley',
 'hyatt place dallas-north',
 'ramada inn boston',
 'grand hyatt san francisco',
 'the whitehall hotel',
 'jameson inn',
 'comfort suites denver international airport',
 'homestead studio suites - dallas - plano',
 'holiday inn express ohio state fair/expo center',
 'jacksonville marriott',
 'springhill suites charlotte airport',
 'inn at the opera',
 'battery wharf hotel, boston waterfront',
 'sterling hotel',
 'la quinta inn houston northwest',
 'hilton garden inn detroit downtown',
 'sleep inn north',
 'crowne plaza charlotte',
 'days inn south seattle',
 'sonesta hotel philadelphia',
 'doubletree suites by hilton - boston',
 'embassy suites washington-convention center',
 'motel 6 columbus']

In [11]:
model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name_or_path = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/models/new_models/tripadvisor/T5_recommender")
model.eval()
model = model.to('cuda')
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base", additional_special_tokens=["computer:", "human:"])
model.resize_token_embeddings(len(tokenizer))
IGNORE_INDEX = -100

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
def chunk(list_of_elements, batch_size): # using this chunk function, we can split our data to multiple batches
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]

def evaluate_recommender(prompt_test, recommend_test, model, tokenizer, batch_size=8, threshold=70):
  prompt_batches = list(chunk(prompt_test, batch_size))
  generation_batches = list(chunk(recommend_test, batch_size))

  correctly_predicted = []
  for prompt_batch, generation_batch in tqdm(zip(prompt_batches, generation_batches), total = len(generation_batches)):

    inputs = tokenizer(prompt_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt") 

    generations_predicted = model.generate(input_ids=inputs["input_ids"].to('cuda'), attention_mask=inputs["attention_mask"].to('cuda'),
                            max_new_tokens=32,
                            num_beams=8,
                            eos_token_id=tokenizer.eos_token_id,
                            pad_token_id=tokenizer.pad_token_id,
                            bos_token_id=tokenizer.bos_token_id) # length_penalty=0.8, Set length_penalty to values < 1.0 in order to encourage the model to generate shorter sequences, to a value > 1.0 in order to encourage the model to produce longer sequences.

    decoded_generations = [tokenizer.decode(generation, skip_special_tokens=True, clean_up_tokenization_spaces=True) for generation in generations_predicted]
    generation_batch = [generation for generation in generation_batch]
    
    correctly_predicted.extend([1 if fuzz.ratio(predicted, ground_truth) > threshold else 0 for predicted, ground_truth in zip(decoded_generations, generation_batch)])

  return correctly_predicted

In [13]:
correctly_predicted = evaluate_recommender(prompt_test, recommend_test, model, tokenizer, batch_size=4, threshold=70)
success_rate = sum(correctly_predicted) / len(correctly_predicted)
print("success_rate: ", success_rate)

100%|██████████| 494/494 [07:07<00:00,  1.15it/s]

success_rate:  0.29295489102889


In [33]:
def chunk(list_of_elements, batch_size): # using this chunk function, we can split our data to multiple batches
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]
    
def convert_to_sublists(numbers, sublist_size):
    return [numbers[i:i+sublist_size] for i in range(0, len(numbers), sublist_size)]

def recommender_rank(prompts, candidate_apps, model, tokenizer, batch_size=8):
  model.eval()
  encoder_max_length = 1024
  decoder_max_length = 32
  prompts_tokenized = tokenizer(prompts, max_length=encoder_max_length, truncation=True, padding="max_length", return_tensors="pt")
  
  input_ids_decoder = []
  attention_mask_decoder = []
  input_ids_encoder = []
  attention_mask_encoder  = []
  for index, candidate_app_elements in enumerate(candidate_apps):
    candidate_app_elements = [tokenizer.pad_token+element for element in candidate_app_elements] # adding pad token to the beginning of each candidate app
    candidate_apps_tokenized = tokenizer(candidate_app_elements, max_length=decoder_max_length, truncation=True, padding="max_length", return_tensors="pt")
    for candidate_app_index in range(len(candidate_app_elements)):
      input_ids_decoder.append(candidate_apps_tokenized["input_ids"][candidate_app_index])
      attention_mask_decoder.append(candidate_apps_tokenized["attention_mask"][candidate_app_index])
      input_ids_encoder.append(prompts_tokenized["input_ids"][index])
      attention_mask_encoder.append(prompts_tokenized["attention_mask"][index])
  
  input_ids_encoder_batches = list(chunk(input_ids_encoder, batch_size))
  attention_mask_encoder_batches = list(chunk(attention_mask_encoder, batch_size))
  input_ids_decoder_batches = list(chunk(input_ids_decoder, batch_size))
  attention_mask_decoder_batches = list(chunk(attention_mask_decoder, batch_size))
  

  scores = []
  for input_ids_encoder_batch, attention_mask_encoder_batch, input_ids_decoder_batch, attention_mask_decoder_batch in tqdm(zip(input_ids_encoder_batches, attention_mask_encoder_batches, input_ids_decoder_batches, attention_mask_decoder_batches), total = len(input_ids_encoder_batches)):
    decoder_input_ids = torch.stack(input_ids_decoder_batch).to("cuda")
    decoder_attention_mask = torch.stack(attention_mask_decoder_batch).to("cuda")
    input_ids = torch.stack(input_ids_encoder_batch).to("cuda")
    attention_mask = torch.stack(attention_mask_encoder_batch).to("cuda")
    with torch.no_grad():
      model_output = model(decoder_input_ids=decoder_input_ids, decoder_attention_mask=decoder_attention_mask, 
                           input_ids=input_ids, attention_mask=attention_mask)
    
    logprobs = F.log_softmax(model_output["logits"], dim=-1)[:, :-1, :] # remove the eos token
    output_tokens = decoder_input_ids[:, 1:] # remove the bos token
        
    tokens_logprobs = torch.gather(logprobs, 2, output_tokens[:, :, None]).squeeze(-1).to(torch.float32)
        
    mask = torch.ones(tokens_logprobs.shape, dtype=torch.bool, device="cuda")
    for i, _output in enumerate(output_tokens):
      for j, _token in enumerate(_output):
        if _token == tokenizer.pad_token_id:
          mask[i, j] = False
              
    score = (tokens_logprobs * mask).sum(-1) / mask.sum(-1)
    scores.extend(score.to('cpu').tolist())
    
  # batch_input_representations = torch.cat(batch_input_representations)
  
  scores = convert_to_sublists(scores, len(candidate_apps[0]))
  
  return scores

In [34]:
scores = recommender_rank(prompt_test, candidate_books, model, tokenizer, batch_size=8)

100%|██████████| 6166/6166 [23:31<00:00,  4.37it/s]


Sampled Candidates

In [30]:
[top_k_accuracy_score(true_candidate_indexes, scores, k=k) for k in range(1, 11)]

[np.float64(0.4368340943683409),
 np.float64(0.563165905631659),
 np.float64(0.6377473363774734),
 np.float64(0.6948249619482496),
 np.float64(0.7404870624048706),
 np.float64(0.7663622526636226),
 np.float64(0.7945205479452054),
 np.float64(0.8249619482496194),
 np.float64(0.8462709284627092),
 np.float64(0.8645357686453576)]

In [31]:
true_relevance = [[1 if item == index else 0 for item in range(len(candidate_books[0]))] for index in true_candidate_indexes]

In [32]:
[ndcg_score(true_relevance, scores, k=k) for k in range(1, 11)]

[np.float64(0.4368340943683409),
 np.float64(0.5165405929169422),
 np.float64(0.5538313082898493),
 np.float64(0.578413303613673),
 np.float64(0.5960778153595425),
 np.float64(0.6052947440974975),
 np.float64(0.6146808425246918),
 np.float64(0.6242840351209088),
 np.float64(0.6306986773420591),
 np.float64(0.6359784001971803)]

Similar Candidates

In [ ]:
[top_k_accuracy_score(true_candidate_indexes, scores, k=k) for k in range(1, 11)]

In [44]:
true_relevance = [[1 if item == index else 0 for item in range(len(candidate_books[0]))] for index in true_candidate_indexes]

In [ ]:
[ndcg_score(true_relevance, scores, k=k) for k in range(1, 11)]